In [1]:
!pip install geopy
!pip install pandas

In [2]:
import pandas as pd
df = pd.read_csv('train/train.csv', index_col=0)
print df.head()

           vendor_id      pickup_datetime     dropoff_datetime  \
id                                                               
id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

           passenger_count  pickup_longitude  pickup_latitude  \
id                                                              
id2875421                1        -73.982155        40.767937   
id2377394                1        -73.980415        40.738564   
id3858529                1        -73.979027        40.763939   
id3504673                1        -74.010040        40.719971   
id2181028                1        -73.973053        40.793209   

           dropoff_longitude  dropoff_latitude store_and_fwd_flag  \
id          

In [3]:
from geopy.distance import great_circle
from dateutil.parser import parse

def get_distance(lat1, long1, lat2, long2):
    return great_circle((lat1, long1), (lat2, long2)).miles

df['distance'] = df.apply(lambda row : get_distance(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], 
                                                    row['dropoff_longitude']), axis=1)
df['month'] = df.apply(lambda row: parse(row['pickup_datetime']).month, axis=1)
df['day'] = df.apply(lambda row: parse(row['pickup_datetime']).day, axis=1)
df['pickup_hour'] = df.apply(lambda row: parse(row['pickup_datetime']).hour, axis=1)
df['dropoff_hour'] = df.apply(lambda row: parse(row['dropoff_datetime']).hour, axis=1)

print df.head()

           vendor_id      pickup_datetime     dropoff_datetime  \
id                                                               
id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

           passenger_count  pickup_longitude  pickup_latitude  \
id                                                              
id2875421                1        -73.982155        40.767937   
id2377394                1        -73.980415        40.738564   
id3858529                1        -73.979027        40.763939   
id3504673                1        -74.010040        40.719971   
id2181028                1        -73.973053        40.793209   

           dropoff_longitude  dropoff_latitude store_and_fwd_flag  \
id          

In [11]:
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor

yes_func = lambda x: 1 if x == 'Y' else 0
customized_df = df[['vendor_id', 'month', 'day', 'pickup_hour', 'dropoff_hour', 'store_and_fwd_flag', 'trip_duration']]
customized_df['store_and_fwd_flag'] = customized_df.apply(lambda row: yes_func(row['store_and_fwd_flag']), axis=1)

x = customized_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled, columns=customized_df.columns)
print scaled_df

C:\Users\t-govenk\AppData\Local\Continuum\Anaconda2\envs\kaggle\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
from sklearn.metrics import accuracy_score

X, y = scaled_df[['vendor_id', 'month', 'day', 'pickup_hour', 'dropoff_hour', 'store_and_fwd_flag']].values, scaled_df[['trip_duration']].values
model = GradientBoostingRegressor()
model.fit(X, y)
predictions = model.predict(X)
print accuracy_score(y, predictions)

ValueError: continuous is not supported